In [1]:
import pandas as pd,numpy as np
import tldextract,re

In [2]:
df_lkdn_cb = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/saas_classification/comps_saas_linkedin_crunchbase_jan04.csv')
df_lkdn = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/saas_classification/comps_saas_specialty_linkedin_jan03.csv')
df_lkdn_website_text1 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/saas_classification/comps_saas_specialty_linkedin_jan03_1to3000_out_all_texts.csv',names=['id','website_text'])
df_lkdn_website_text2 = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/saas_classification/comps_saas_specialty_linkedin_jan03_3001to6000_out_all_texts.csv',names=['id','website_text'])
df_cb = pd.read_excel('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/saas_classification/USA_SAAS_CB_10Dec16.xlsx')
df_cb_website_text = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/saas_classification/USA_SAAS_CB_10Dec16_website_text.csv')

In [3]:
#get domains so that we can use it to join
df_lkdn_cb['domain_cleaned'] = df_lkdn_cb['id'].apply(lambda x: tldextract.extract(x.lower()).domain)
df_lkdn['domain_cleaned'] = df_lkdn['id'].apply(lambda x: tldextract.extract(x.lower()).domain)
df_lkdn_website_text1['domain_cleaned'] = df_lkdn_website_text1['id'].apply(lambda x: tldextract.extract(x.lower()).domain)
df_lkdn_website_text2['domain_cleaned'] = df_lkdn_website_text2['id'].apply(lambda x: tldextract.extract(x.lower()).domain)
df_cb['domain_cleaned'] = df_cb['homepage_url'].apply(lambda x: tldextract.extract(x.lower()).domain)
df_cb_website_text['domain_cleaned'] = df_cb_website_text['website'].apply(lambda x: tldextract.extract(x.lower()).domain)

No handlers could be found for logger "tldextract"


In [4]:
#combining website text dfs
df_cb_website_text.columns = ['id','website_text','domain_cleaned']
df_website_texts = pd.concat([df_lkdn_website_text1,df_lkdn_website_text2,df_cb_website_text],ignore_index=True)
df_website_texts['domain_cleaned'] = df_website_texts['domain_cleaned'].str.lower()
df_lkdn['domain_cleaned'] = df_lkdn['domain_cleaned'].str.lower()

In [5]:
#combining other dfs
df_lkdn_tmp = pd.merge(df_lkdn[['domain_cleaned','description','specialties','industry']],
               df_lkdn_cb[['domain_cleaned','description','short_description','category_list','category_group_list']],
                       on='domain_cleaned',how='left')
df_lkdn_tmp['descriptions'] = df_lkdn_tmp[['description_x','specialties','description_y','short_description']].fillna('').\
    apply(func=lambda x:' '.join([i for i in x if i]),axis=1)
df_cb['descriptions'] = df_cb[['short_description','description']].fillna('').apply(func=lambda x:' '.join([i for i in x if i]),axis=1)
df_lkdn_tmp['categories'] = df_lkdn_tmp[['category_list','category_group_list']].fillna('').\
    apply(func=lambda x:'|'.join([i for i in x if i]),axis=1)
df_cb['categories'] = df_cb[['category_list','category_group_list']].fillna('').\
    apply(func=lambda x:'|'.join([i for i in x if i]),axis=1)
df_descr_categories = pd.concat([df_lkdn_tmp[['domain_cleaned','descriptions','categories']],
                                df_cb[['domain_cleaned','descriptions','categories']]],ignore_index=True)

In [6]:
df_final =  pd.merge(df_descr_categories,df_website_texts[['domain_cleaned','website_text']],on='domain_cleaned')
df_final['website_text'] = df_final['website_text'].fillna('').apply(lambda x: re.sub('\n',' ',x))
df_final.shape

/usr/local/lib/python2.7/dist-packages/pandas/tools/merge.py:1323: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  llab = rizer.factorize(lk)


(7325, 4)

In [7]:
df_final = df_final.drop_duplicates('domain_cleaned')
df_final.shape

(5322, 4)

In [8]:
#checking if any row with only space in categories
print df_final['categories'].str.contains(r'^ *$').value_counts()
print (df_final['categories']=='').value_counts()

False    2679
True     2643
Name: categories, dtype: int64
False    2679
True     2643
Name: categories, dtype: int64


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegressionCV,LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [10]:
df_final_labelled = df_final[df_final['categories']!='']
df_final_unlabelled = df_final[df_final['categories']=='']


In [11]:
def category_tokenizer(text):
    ''' split by pipe (|) and return'''
    text = text.strip()
    text = re.sub('\|$|^\|','',text)
    if text == '|':
        return []
    return [i.strip() for i in text.lower().split('|')]
vectorizer = CountVectorizer(max_df=1.0, min_df=50,tokenizer=category_tokenizer,binary=True)
dv_matrix_labelled = vectorizer.fit_transform(df_final_labelled['categories'].fillna(''))
dv_matrix_labelled = dv_matrix_labelled.toarray()
dv_matrix_labelled.shape

(2679, 50)

In [12]:
dv_matrix_unlabelled = np.zeros((df_final_unlabelled.shape[0],dv_matrix_labelled.shape[1])) - 1
dv_matrix_unlabelled.shape

(2643, 50)

In [13]:
dv_matrix = np.concatenate([dv_matrix_labelled,dv_matrix_unlabelled])

In [14]:
tfidf_vectorizer_descr = TfidfVectorizer(max_df=0.8, min_df=0.01,stop_words='english')
X_descr_train = tfidf_vectorizer_descr.fit_transform(df_final_labelled['descriptions'].append(df_final_unlabelled['descriptions'],ignore_index=True).fillna(''))

tfidf_vectorizer_website_text = TfidfVectorizer(max_df=0.8, min_df=0.01,stop_words='english')
X_website_text_train = tfidf_vectorizer_website_text.fit_transform(df_final_labelled['website_text'].append(df_final_unlabelled['website_text'],ignore_index=True).fillna(''))
dv_matrix.shape,X_descr_train.shape,X_website_text_train.shape

((5322, 50), (5322, 1258), (5322, 6259))

In [16]:
del df_cb,df_cb_website_text,df_descr_categories,df_final,df_final_labelled,df_final_unlabelled,df_lkdn,df_lkdn_cb,df_lkdn_tmp,df_lkdn_website_text1,df_lkdn_website_text2,df_website_texts

NameError: name 'df_final' is not defined

In [24]:
import cotraining_multi
print 'logistic regression multi'
clf = cotraining_multi.CotrainMulti() #MultinomialNB(fit_prior=False)
clf.fit(LogisticRegressionCV(class_weight='balanced',Cs=10,scoring='f1',n_jobs=1),
        X_descr_train,X_website_text_train,dv_matrix,k=1000,u=dv_matrix_unlabelled.shape[0]//10)

logistic regression multi


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


no of iterations took for fitting:75
no of iterations took for fitting:176
no of iterations took for fitting:79
no of iterations took for fitting:65
no of iterations took for fitting:106
no of iterations took for fitting:64
no of iterations took for fitting:182
no of iterations took for fitting:54
no of iterations took for fitting:176
no of iterations took for fitting:51
no of iterations took for fitting:65
no of iterations took for fitting:85
no of iterations took for fitting:56
no of iterations took for fitting:53
no of iterations took for fitting:282
no of iterations took for fitting:68
no of iterations took for fitting:49
no of iterations took for fitting:41
no of iterations took for fitting:132
no of iterations took for fitting:64
no of iterations took for fitting:367
no of iterations took for fitting:36
no of iterations took for fitting:136
no of iterations took for fitting:151
no of iterations took for fitting:87
no of iterations took for fitting:65
no of iterations took for fit

In [28]:
dv_labels = range(len(vectorizer.vocabulary_))
for key in vectorizer.vocabulary_:
    dv_labels[vectorizer.vocabulary_[key]] = key

In [30]:
import pickle
with open('saas_multi_model_test_object.pkl','w') as f:
    pickle.dump({'model':clf,'website_text_vectorizer':tfidf_vectorizer_website_text,
                'description_vectorizer':tfidf_vectorizer_descr,'dv_labels':dv_labels},f)

In [29]:
dv_labels

[u'advertising',
 u'analytics',
 u'apps',
 u'b2b',
 u'big data',
 u'business intelligence',
 u'cloud computing',
 u'collaboration',
 u'commerce and shopping',
 u'consulting',
 u'content and publishing',
 u'crm',
 u'curated web',
 u'customer service',
 u'data and analytics',
 u'design',
 u'developer tools',
 u'digital media',
 u'e-commerce',
 u'education',
 u'enterprise software',
 u'finance',
 u'financial services',
 u'hardware',
 u'health care',
 u'human resources',
 u'information technology',
 u'internet',
 u'internet services',
 u'marketing automation',
 u'media and entertainment',
 u'messaging and telecommunications',
 u'mobile',
 u'paas',
 u'payments',
 u'privacy and security',
 u'professional services',
 u'project management',
 u'real estate',
 u'recruiting',
 u'saas',
 u'sales and marketing',
 u'science and engineering',
 u'security',
 u'small and medium businesses',
 u'social media',
 u'software',
 u'transportation',
 u'video',
 u'web development']

### getting predictions for unlabelled

In [11]:
import pickle
with open('saas_multi_model_test_object.pkl','r') as f:
    tmp = pickle.load(f)

model,website_text_vectorizer,description_vectorizer,dv_labels = tmp['model'],tmp['website_text_vectorizer'],tmp['description_vectorizer'],tmp['dv_labels']
del tmp

In [14]:
X_descr = description_vectorizer.transform(df_final_unlabelled['descriptions'])
X_website_text = website_text_vectorizer.transform(df_final_unlabelled['website_text'])
X_descr.shape,X_website_text.shape

((2643, 1258), (2643, 6259))

In [15]:
preds = model.predict(X_descr,X_website_text)
preds_df = pd.DataFrame(preds,columns=dv_labels)
preds_df.head()

,advertising,analytics,apps,b2b,big data,business intelligence,cloud computing,collaboration,commerce and shopping,consulting,...,saas,sales and marketing,science and engineering,security,small and medium businesses,social media,software,transportation,video,web development
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [41]:
df_final_unlabelled.index = range(df_final_unlabelled.shape[0])
df_unlab_preds = pd.concat([df_final_unlabelled[['domain_cleaned','descriptions']],preds_df],axis=1)

In [43]:
df_unlab_preds.head()

,domain_cleaned,descriptions,advertising,analytics,apps,b2b,big data,business intelligence,cloud computing,collaboration,...,saas,sales and marketing,science and engineering,security,small and medium businesses,social media,software,transportation,video,web development
0,1С-edi,"1C Net (""1C-Сеть"" in Russian) was founded by R...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020thinkology,We find future success for our clients. Four k...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2boandco,Société de conseil en solutions SAAS (Software...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,360medical,"> 360 medics, la 1ère application sur le bon u...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,4clouds,"We design, configure, deploy and maintain netw...",0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [44]:
df_unlab_preds.to_csv('saas_model_predictions.csv',index=False,quoting=1,encoding='utf-8')

In [42]:
df_unlab_preds.shape

(2643, 52)